# SUM Experiments Using Synthetic Dataset

In [1]:
import sys
from os import path
sys.path.insert(0, '../src')
import one_to_n
sys.path.insert(0, '../tests')
import create_synthetic_dataset
import datetime
import textdistance
import editdistance
import pandas as pd
import networkx as nx
import re

import editdistance
from Matching import core2
from Matching import analyze
from Matching import matcher
import sys
import os
import datetime

### Create Synthetic Dataset

In [30]:
# Table A later gets duplicated in the bipartite graph maker function.
table_a_non_duplicated = create_synthetic_dataset.create_first_df(100)

table_b_non_typo = create_synthetic_dataset.create_second_df(100)

table_b = create_synthetic_dataset.add_typo(table_a_non_duplicated, table_b_non_typo)

print(table_a_non_duplicated)
print(table_b)

table_a_non_duplicated.to_csv(r's_data1.csv', index = False, header=True)
table_b.to_csv(r's_data2.csv', index = False, header=True)

                name age
0        Cody Ortega   4
1        Rick Martin   7
2        Frank Patel   9
3    Matthew Higgins   5
4   Mr. Johnny Allen   6
..               ...  ..
95     Mitchell Meza   2
96      David Orozco   0
97     Michael Scott   9
98       Andrew Rice   1
99     Nathan Vargas   5

[100 rows x 2 columns]
                  name  age
0         Cody OrtegaK    1
1        Cody OrtegatZ    0
2       Rick MartinnHo    6
3       Frank Patelflf    4
4   Matthew HigginsfyI    1
..                 ...  ...
95    Justin Durhamwvz    0
96    Mitchell Mezaofq    0
97     David OrozcogBK    4
98    Michael ScottzSU    8
99      Andrew RicegpG    7

[100 rows x 2 columns]


### 1. Bipartite Matching

In [31]:
now = datetime.datetime.now()
bipartite_graph_result = one_to_n.keycomp_treshold_updated_maximal_construct_graph(table_a_non_duplicated, table_b, "name", 0.09)
timing_tresh = (datetime.datetime.now()-now).total_seconds()
print("---- Timing for Graph Construction with Treshold Constraint ----")
print(timing_tresh,"seconds")

---- Timing for Graph Construction with Treshold Constraint ----
0.271883 seconds


In [32]:
def SUM_edit_edge_weight(bip_graph):
    for u,v,d in bip_graph.edges(data=True):
        val_tuple_1 = u.split("_")
        val_tuple_2 = v.split("_")
        
        if len(val_tuple_1) == 4:
            val1 = re.sub("[^0-9]", "", val_tuple_1[2])
        else: 
            val1 = re.sub("[^0-9]", "", val_tuple_1[1])
            
        if len(val_tuple_2) == 4:
            val2 =re.sub("[^0-9]", "", val_tuple_2[2])
        else:
            val2 =re.sub("[^0-9]", "", val_tuple_2[1])

        d['weight'] = float(val1) + float(val2)

    return bip_graph

sum_weighted_graph = SUM_edit_edge_weight(bipartite_graph_result)

## Maximal Matching

In [33]:
print("\n\n 'SUM' MAXIMAL MATCHING:")
now = datetime.datetime.now()
matching_set_maximal = nx.algorithms.matching.max_weight_matching(sum_weighted_graph)
timing_match = (datetime.datetime.now()-now).total_seconds()
print("The Maximal Matching Set is:", matching_set_maximal, "\n")
print("---- Timing for Matching (Done on the graph constructed with the treshold constraint) ----")
print(timing_match,"seconds")
print("The number of edges in the graph is:", sum_weighted_graph.number_of_edges(), "\n")


# print("The Maximal Matching Set is:", matching_set_maximal, "\n")



 'SUM' MAXIMAL MATCHING:
The Maximal Matching Set is: {('Cameron MitchellDwL_(1,)_2', 'Cameron Mitchell_0_(4,)_1'), ('Kathy Black_1_(8,)_1', 'Juan Whitenak_(1,)_2'), ('Jamie Jackson_2_(6,)_1', 'Martin WatsontJQ_(4,)_2'), ('Matthew Johnsonnyk_(0,)_2', 'Anna Johnson_2_(6,)_1'), ('Heidi Powers_0_(8,)_1', 'Heidi PowersWVa_(0,)_2'), ('Anthony Sanchez_1_(8,)_1', 'Jonathan LynchRig_(7,)_2'), ('Johnny Harvey_2_(7,)_1', 'John EnglishfCi_(9,)_2'), ('Tina Taylor_0_(9,)_1', 'Tim PetersozP_(7,)_2'), ('Rebecca LarsenZIr_(0,)_2', 'Rebecca Larsen_0_(7,)_1'), ('Cameron Mitchell_2_(4,)_1', 'Kimberly SmithxCQ_(3,)_2'), ('Daryl Fernandez_2_(8,)_1', 'Carrie JordanvCe_(5,)_2'), ('Tara GlassJTW_(5,)_2', 'Kathy Black_2_(8,)_1'), ('Steven GatesdUF_(8,)_2', 'Steven Griffin_2_(5,)_1'), ('Andrew Simmons_2_(4,)_1', 'Andrew Simmonsqoz_(4,)_2'), ('Samantha Williams_2_(7,)_1', 'Samantha WilliamsNTL_(8,)_2'), ('Ian DavisjlY_(3,)_2', 'Tara Glass_0_(9,)_1'), ('Rick Martin_0_(7,)_1', 'Valerie MataLnU_(8,)_2'), ('Sarah 

## Minimal Matching

In [34]:
def minimal_matching(sum_weighted_graph):

    new_graph = sum_weighted_graph.copy()
    max_weight = max([d['weight'] for u,v,d in new_graph.edges(data=True)])
    for u,v,d in new_graph.edges(data=True):
        d['weight'] = max_weight - d['weight']

    matching_set_minimal = nx.algorithms.matching.max_weight_matching(new_graph)
    return matching_set_minimal

In [35]:
print("\n\n 'SUM' MINIMAL MATCHING RESULTS:")
print(nx.bipartite.is_bipartite(sum_weighted_graph))
now = datetime.datetime.now()
matching_set_minimal = minimal_matching(sum_weighted_graph)
timing_match = (datetime.datetime.now()-now).total_seconds()
print("The Minimal Matching Set is:", matching_set_minimal, "\n")
print("---- Timing for Matching (Done on the graph constructed with the treshold constraint) ----")
print(timing_match,"seconds")
print("The number of edges in the graph is:", sum_weighted_graph.number_of_edges(), "\n")



 'SUM' MINIMAL MATCHING RESULTS:
True
The Minimal Matching Set is: {('Anna Williams_1_(2,)_1', 'Samantha WilliamsNTL_(8,)_2'), ('Anna Williams_0_(2,)_1', 'Lisa FisheraRP_(9,)_2'), ('Steven Gates_2_(2,)_1', 'Jesus CartersWH_(5,)_2'), ('Matthew AverynsP_(6,)_2', 'Matthew Avery_1_(2,)_1'), ('Kimberly SmithxCQ_(3,)_2', 'Kimberly Smith_2_(1,)_1'), ('Patricia Lopez_2_(2,)_1', 'Maria SolisEOe_(0,)_2'), ('Lori Sullivan_1_(2,)_1', 'Lori Sullivanqym_(9,)_2'), ('Sergio RodriguezhYm_(3,)_2', 'Kari Rodriguez_2_(5,)_1'), ('Maria Stephens_0_(0,)_1', 'Jason FostergRS_(5,)_2'), ('Anthony SanchezarI_(9,)_2', 'Anthony Tate_2_(4,)_1'), ('Jonathan Lynch_2_(3,)_1', 'Johnny HarveyxjF_(2,)_2'), ('Maria Stephens_1_(0,)_1', 'Martin WatsontJQ_(4,)_2'), ('Matthew Avery_2_(2,)_1', 'Matthew Johnsonnyk_(0,)_2'), ('Amy JohnsonsJg_(3,)_2', 'Matthew Johnson_2_(1,)_1'), ('Jessica Cain_2_(6,)_1', 'Jessica CainjfT_(2,)_2'), ('Maria Stephens_2_(0,)_1', 'Mark MathewsbeR_(0,)_2'), ('Emily LynchYPF_(5,)_2', 'Emily Lynch_2_(

In [36]:
def fetch_sum(bip_graph, matching):
    output = []
    for u,v,d in bip_graph.edges(data=True):
        l = (u, v)
        k = (v, u)
        if l in matching:
            output.append([u,v, d['weight']])
        if k in matching:
            output.append([v,u, d['weight']])
    return output

out_max = fetch_sum(sum_weighted_graph, matching_set_maximal)
out_min = fetch_sum(sum_weighted_graph, matching_set_minimal)

print("\n MAX MATCHES WITH SUM WEIGHTS")
for val in out_max:
    print(val)
    
print("\n MIN MATCHES WITH SUM WEIGHTS")
for val2 in out_min:
    print(val2)

def formatted_output(out_max, out_min):
    out_dict = {}
    for (val1,val2, weight) in out_min:
        splitted1 = val1.split("_")
        splitted2 = val2.split("_")
        if len(splitted1) == 4:
            if splitted1[0] in out_dict:
                out_dict[splitted1[0]].append((splitted2[0], "min", weight))
            else:
                out_dict[splitted1[0]] = [(splitted2[0], "min", weight)]

        if len(splitted2) == 4:
            if splitted2[0] in out_dict:
                out_dict[splitted2[0]].append((splitted1[0], "min", weight))
            else:
                out_dict[splitted2[0]] = [(splitted1[0], "min", weight)]
            
    for (val1,val2, weight) in out_max:
        splitted1 = val1.split("_")
        splitted2 = val2.split("_")
        if len(splitted1) == 4:
            if splitted1[0] in out_dict:
                out_dict[splitted1[0]].append((splitted2[0], "max", weight))
            else:
                out_dict[splitted1[0]] = [(splitted2[0], "max", weight)]

        if len(splitted2) == 4:
            if splitted2[0] in out_dict:
                out_dict[splitted2[0]].append((splitted1[0], "max", weight))
            else:
                out_dict[splitted2[0]] = [(splitted1[0], "max", weight)]
    return out_dict
            
    
form_output = formatted_output(out_max,out_min)

print("\n FORMAL OUTPUT")
for i,val in form_output.items():
    print(i, val)


 MAX MATCHES WITH SUM WEIGHTS
['Cody Ortega_0_(4,)_1', 'Cody AndersonPec_(8,)_2', 12.0]
['Johnny Harvey_0_(7,)_1', 'Cody OrtegaK_(1,)_2', 8.0]
['Cody OrtegatZ_(0,)_2', 'Nancy Brennan_0_(9,)_1', 9.0]
['Leah PoolepHm_(2,)_2', 'Heidi Powers_1_(8,)_1', 10.0]
['Kathy Black_1_(8,)_1', 'Juan Whitenak_(1,)_2', 9.0]
['Jesus Carter_0_(9,)_1', 'Jason FostergRS_(5,)_2', 14.0]
['Heidi Powers_0_(8,)_1', 'Heidi PowersWVa_(0,)_2', 8.0]
['Anthony Sanchez_0_(8,)_1', 'Anthony Tatevsg_(3,)_2', 11.0]
['Rhonda Bell_2_(7,)_1', 'Ryan YangeYy_(7,)_2', 14.0]
['Andrew RicegpG_(7,)_2', 'Matthew Higgins_2_(5,)_1', 12.0]
['Rick Martin_0_(7,)_1', 'Valerie MataLnU_(8,)_2', 15.0]
['Frank Patel_0_(9,)_1', 'Rick MartinnHo_(6,)_2', 15.0]
['Frank Patelflf_(4,)_2', 'Frank Patel_1_(9,)_1', 13.0]
['Tina TaylorDBD_(7,)_2', 'Tina Taylor_1_(9,)_1', 16.0]
['Rick Martin_1_(7,)_1', 'Tonya MartinezmjF_(3,)_2', 10.0]
['Tara Glass_1_(9,)_1', 'Tina Davislid_(0,)_2', 9.0]
['Ian DavisjlY_(3,)_2', 'Tara Glass_0_(9,)_1', 12.0]
['Rick Mar

## BP Matching: Sum row of Min and Max

In [37]:
def sum_total_weights(max_min_list):
    if max_min_list == [] or max_min_list == None:
        print("ERROR: NO SIMILARITY FOUND IN NAIVE OR RANDOM SAMPLING APPROACH. Suggestion: Decrease Similarity Matching Threshold.")
        return None
    total = 0
    for i in max_min_list:
        total += i[-1]
    return total

total_max = sum_total_weights(out_max)
print("BP Matching: Highest bound for maximum:", total_max)

total_min = sum_total_weights(out_min)
print("BP Matching: Lowest bound for minimum:", total_min)

BP Matching: Highest bound for maximum: 1055.0
BP Matching: Lowest bound for minimum: 641.0


### Prepare Data for Comparison Experimentation

In [38]:
# print(os.getcwd())
sample_size = 50
table_a_dup = one_to_n.create_duplicates(table_a_non_duplicated, "name", 3)
table_a_dup.to_csv(r's_data1_dup.csv', index = False, header=True)
cat_table1_dup = core2.data_catalog('s_data1_dup.csv')
cat_table1 = core2.data_catalog('s_data1.csv')
cat_table2 = core2.data_catalog('s_data2.csv')
print('Loaded catalogs.')

Loaded catalogs.


### 2. Naive Matching Performance Evaluation

In [39]:
# MAX MATCHING
print("NAIVE MAX MATCHING")
print('Performing compare all match (edit distance)...')
now = datetime.datetime.now()
max_compare_all_edit_match = matcher.matcher_dup_updated(cat_table1_dup,cat_table2,editdistance.eval, matcher.all, 0.09)
naive_time_edit = (datetime.datetime.now()-now).total_seconds()
print("Naive Edit Distance Matching computation time taken: ", naive_time_edit, " seconds")
#print('Compare All Matcher (Edit Distance) Performance: ' + str(core.eval_matching(compare_all_edit_match)))


print('Performing compare all match (jaccard distance)...')
now = datetime.datetime.now()
max_compare_all_jaccard_match = matcher.matcher_dup_updated(cat_table1_dup,cat_table2,analyze.jaccard_calc, matcher.all, 0.09)
naive_time_jaccard = (datetime.datetime.now()-now).total_seconds()
print("Naive Jaccard Matching computation time taken: ", naive_time_jaccard, " seconds", "\n")
#print('Compare All Matcher (Jaccard Distance) Performance: ' + str(core2.eval_matching(compare_all_jaccard_match)))

# MIN MATCHING
print("NAIVE MIN MATCHING")
print('Performing compare all match (edit distance)...')
now = datetime.datetime.now()
min_compare_all_edit_match = matcher.matcher_updated(cat_table1,cat_table2,editdistance.eval, matcher.all, 0.09)
naive_time_edit = (datetime.datetime.now()-now).total_seconds()
print("Naive Edit Distance Matching computation time taken: ", naive_time_edit, " seconds")
#print('Compare All Matcher (Edit Distance) Performance: ' + str(core.eval_matching(compare_all_edit_match)))


print('Performing compare all match (jaccard distance)...')
now = datetime.datetime.now()
min_compare_all_jaccard_match = matcher.matcher_updated(cat_table1,cat_table2,analyze.jaccard_calc, matcher.all, 0.09)
naive_time_jaccard = (datetime.datetime.now()-now).total_seconds()
print("Naive Jaccard Matching computation time taken: ", naive_time_jaccard, " seconds")
#print('Compare All Matcher (Jaccard Distance) Performance: ' + str(core2.eval_matching(compare_all_jaccard_match)))




NAIVE MAX MATCHING
Performing compare all match (edit distance)...
Naive Edit Distance Matching computation time taken:  0.218585  seconds
Performing compare all match (jaccard distance)...
Naive Jaccard Matching computation time taken:  0.142916  seconds 

NAIVE MIN MATCHING
Performing compare all match (edit distance)...
Naive Edit Distance Matching computation time taken:  0.077867  seconds
Performing compare all match (jaccard distance)...
Naive Jaccard Matching computation time taken:  0.046205  seconds


### 3. Random Sampling Matching Performance Evaluation

In [40]:
# MAX MATCHING
print("RANDOM SAMPLE MAX MATCHING")
print('Performing random sample match (edit distance)...')
now = datetime.datetime.now()
max_compare_sampled_edit_match = matcher.matcher_dup_updated(cat_table1_dup,cat_table2,editdistance.eval, matcher.random_sample, 0.09, sample_size)
sim_time_edit = (datetime.datetime.now()-now).total_seconds()
print("Simulation-Based Edit Distance Matching computation time taken: ", sim_time_edit, " seconds")
#print('Random Sample Matcher (Edit Distance) Performance: ' + str(core.eval_matching(compare_all_edit_match)))

# print('Performing random sample match (jaccard distance)...')
# now = datetime.datetime.now()
# max_compare_sampled_jaccard_match = matcher.matcher_dup_updated(cat_table1_dup,cat_table2,analyze.jaccard_calc, matcher.random_sample, 0.05, sample_size)
# sim_time_jaccard = (datetime.datetime.now()-now).total_seconds()
# print("Simulation-Based Jaccard Matching computation time taken: ", sim_time_jaccard, " seconds", "\n")
#print('Random Sample Matcher (Jaccard Distance) Performance: ' + str(core.eval_matching(compare_all_jaccard_match)))


# MIN MATCHING
print("RANDOM SAMPLE MIN MATCHING")
print('Performing random sample match (edit distance)...')
now = datetime.datetime.now()
min_compare_sampled_edit_match = matcher.matcher_updated(cat_table1,cat_table2,editdistance.eval, matcher.random_sample, 0.09, sample_size)
sim_time_edit = (datetime.datetime.now()-now).total_seconds()
print("Simulation-Based Edit Distance Matching computation time taken: ", sim_time_edit, " seconds")
#print('Random Sample Matcher (Edit Distance) Performance: ' + str(core.eval_matching(compare_all_edit_match)))

# print('Performing random sample match (jaccard distance)...')
# now = datetime.datetime.now()
# min_compare_sampled_jaccard_match = matcher.matcher_updated(cat_table1,cat_table2,analyze.jaccard_calc, matcher.random_sample, 0.5, sample_size)
# sim_time_jaccard = (datetime.datetime.now()-now).total_seconds()
# print("Simulation-Based Jaccard Matching computation time taken: ", sim_time_jaccard, " seconds")
#print('Random Sample Matcher (Jaccard Distance) Performance: ' + str(core.eval_matching(compare_all_jaccard_match)))

RANDOM SAMPLE MAX MATCHING
Performing random sample match (edit distance)...
Simulation-Based Edit Distance Matching computation time taken:  0.182  seconds
RANDOM SAMPLE MIN MATCHING
Performing random sample match (edit distance)...
Simulation-Based Edit Distance Matching computation time taken:  0.056253  seconds


In [47]:
naive_total_max = sum_total_weights(max_compare_all_edit_match)
sampled_total_max = sum_total_weights(max_compare_sampled_edit_match)
bp_total_max = sum_total_weights(out_max)


print("MAX Matching Bound:")
print("BP Matching: ", total_max)
print("NAIVE Matching: ", naive_total_max)
print("SAMPLED Matching: ", sampled_total_max, "\n")

naive_total_min = sum_total_weights(min_compare_all_edit_match)
sampled_total_min = sum_total_weights(min_compare_sampled_edit_match)
bp_total_min = sum_total_weights(out_min)


print("MIN Matching Bound:")
print("BP Matching: ", total_min)
print("NAIVE Matching: ", naive_total_min)
print("SAMPLED Matching: ", sampled_total_min, "\n")


print("MAX Count of Matches:")
print("BP Matching Count:", len(out_max))
print("NAIVE Matching Count: ", len(max_compare_all_edit_match))
print("SAMPLED Matching Count:", len(max_compare_sampled_edit_match), "\n")

print("MIN Count of Matches:")
print("BP Matching Count:", len(out_min))
print("NAIVE Matching Count: ", len(min_compare_all_edit_match))
print("SAMPLED Matching Count:", len(min_compare_sampled_edit_match), "\n")

MAX Matching Bound:
BP Matching:  1055.0
NAIVE Matching:  12393
SAMPLED Matching:  5974 

MIN Matching Bound:
BP Matching:  641.0
NAIVE Matching:  4131
SAMPLED Matching:  1824 

MAX Count of Matches:
BP Matching Count: 100
NAIVE Matching Count:  1473
SAMPLED Matching Count: 713 

MIN Count of Matches:
BP Matching Count: 100
NAIVE Matching Count:  491
SAMPLED Matching Count: 222 



In [42]:
# print(min_compare_all_jaccard_match, "\n")

print(max_compare_all_edit_match)
# print(table_a_non_duplicated)
# print(table_b)
# print(min_compare_all_edit_match)

[('Cody Ortega_0', 'Cody OrtegaK', 5), ('Cody Ortega_0', 'Cody OrtegatZ', 4), ('Cody Ortega_0', 'Cody AndersonPec', 12), ('Cody Ortega_0', 'Juan Whitenak', 5), ('Cody Ortega_0', 'Anthony Tatevsg', 7), ('Cody Ortega_0', 'Ryan YangeYy', 11), ('Cody Ortega_0', 'Andrew RicegpG', 11), ('Rick Martin_0', 'Cody OrtegaK', 8), ('Rick Martin_0', 'Rick MartinnHo', 13), ('Rick Martin_0', 'Tina TaylorDBD', 14), ('Rick Martin_0', 'Tonya MartinezmjF', 10), ('Rick Martin_0', 'Juan Whitenak', 8), ('Rick Martin_0', 'Tina Davislid', 7), ('Rick Martin_0', 'Ronald MortonQTl', 11), ('Rick Martin_0', 'Ryan YangeYy', 14), ('Frank Patel_0', 'Cody OrtegaK', 10), ('Frank Patel_0', 'Frank Patelflf', 13), ('Frank Patel_0', 'Evan WeberZlI', 12), ('Frank Patel_0', 'Leah PoolepHm', 11), ('Frank Patel_0', 'Juan Whitenak', 10), ('Frank Patel_0', 'Tina Davislid', 9), ('Frank Patel_0', 'Ian DavisjlY', 12), ('Frank Patel_0', 'Sarah PowellSHT', 10), ('Frank Patel_0', 'Anthony Tatevsg', 12), ('Frank Patel_0', 'Mark Mathewsbe

## Formal Output of Experiment Matches (Naive and Random Sampling)

In [43]:

def formatted_output_other(out_max, out_min):
    out_dict = {}
    for (val1,val2, weight) in out_min:
        splitted1 = val1.split("_")
        if splitted1[0] in out_dict:
            out_dict[splitted1[0]].append((val2, "min", weight))
        else:
            out_dict[splitted1[0]] = [(val2, "min", weight)]

    for (val1,val2, weight) in out_max:
        splitted1 = val1.split("_")
        if splitted1[0] in out_dict:
            out_dict[splitted1[0]].append((val2, "min", weight))
        else:
            out_dict[splitted1[0]] = [(val2, "min", weight)]

    return out_dict
            

In [44]:
form_output_all = formatted_output_other(max_compare_all_edit_match,min_compare_all_edit_match)
form_output_sampled = formatted_output_other(max_compare_sampled_edit_match,min_compare_sampled_edit_match)

In [45]:
for i in form_output_all.items():
    print(i, "\n\n")


('Cody Ortega', [('Cody OrtegaK', 'min', 5), ('Cody OrtegatZ', 'min', 4), ('Cody AndersonPec', 'min', 12), ('Juan Whitenak', 'min', 5), ('Anthony Tatevsg', 'min', 7), ('Ryan YangeYy', 'min', 11), ('Andrew RicegpG', 'min', 11), ('Cody OrtegaK', 'min', 5), ('Cody OrtegatZ', 'min', 4), ('Cody AndersonPec', 'min', 12), ('Juan Whitenak', 'min', 5), ('Anthony Tatevsg', 'min', 7), ('Ryan YangeYy', 'min', 11), ('Andrew RicegpG', 'min', 11), ('Cody OrtegaK', 'min', 5), ('Cody OrtegatZ', 'min', 4), ('Cody AndersonPec', 'min', 12), ('Juan Whitenak', 'min', 5), ('Anthony Tatevsg', 'min', 7), ('Ryan YangeYy', 'min', 11), ('Andrew RicegpG', 'min', 11), ('Cody OrtegaK', 'min', 5), ('Cody OrtegatZ', 'min', 4), ('Cody AndersonPec', 'min', 12), ('Juan Whitenak', 'min', 5), ('Anthony Tatevsg', 'min', 7), ('Ryan YangeYy', 'min', 11), ('Andrew RicegpG', 'min', 11)]) 


('Rick Martin', [('Cody OrtegaK', 'min', 8), ('Rick MartinnHo', 'min', 13), ('Tina TaylorDBD', 'min', 14), ('Tonya MartinezmjF', 'min', 10)